# March Madness ML Algorithm

## Setup

We download libraries and import configurables.

In [4]:
from dotenv import dotenv_values
import logging
import pandas as pd

config = dotenv_values(".env")
logging.getLogger().setLevel(logging.INFO) # toggle logging level

## Load Data

### March Madness results

We import data from every tournament game from 1985 - 2019. This data is stored locally in `data/mm-results.csv`.

In [5]:
df = pd.read_csv('data/mm-results.csv')

### Yearly team stats

We import data on team stats for every season (1985 - 2019). This data is scraped from Sports Reference. If data is not already scraped, run `helper/pull-sports-reference.py`.

INFO:root:Retrieved schedule for Marshall in 1985
INFO:root:Retrieved schedule for Temple in 1985
INFO:root:Retrieved schedule for UNLV in 1985
INFO:root:Retrieved schedule for Mercer in 1985
INFO:root:Retrieved schedule for Navy in 1985
INFO:root:Retrieved schedule for Oklahoma in 1985
INFO:root:Retrieved schedule for Alabama in 1985
INFO:root:Retrieved schedule for North Carolina in 1985
INFO:root:Retrieved schedule for Iowa St in 1985
INFO:root:Retrieved schedule for USC in 1985
ERROR:root:Error retrieving schedule for Fairleigh Dickinson in 1985
INFO:root:Retrieved schedule for Pennsylvania in 1985
INFO:root:Retrieved schedule for Louisiana Tech in 1985
INFO:root:Retrieved schedule for Ohio in 1985
INFO:root:Retrieved schedule for Purdue in 1985
INFO:root:Retrieved schedule for Oregon St in 1985
INFO:root:Retrieved schedule for Wichita St in 1985
INFO:root:Retrieved schedule for Northeastern in 1985
INFO:root:Retrieved schedule for Michigan in 1985
INFO:root:Retrieved schedule for 

## Data Preparation

### Clean up main dataframe

We add, remove, and reorder columns in the main dataframe.

In [7]:
# remove columns that are not needed
columns_to_remove = [
    'Region Number',
    'Region Name',
]
df.drop(columns_to_remove, axis='columns')

# reorder columns
df = df[['Year', 'Round', 'Team 1', 'Seed 1', 'Team 2', 'Seed 2', 'Score 1', 'Score 2']]

# add column for winner
def get_winner(score1, score2):
    if score1 > score2:
        return 1
    elif score1 < score2:
        return 2
    else:
        logging.error(f"Score 1: {score1}, Score 2: {score2}, no winner found")
        return 0
df.insert(8, 'Winner', [get_winner(row['Score 1'], row['Score 2']) for index, row in df.iterrows()])
